Here, we'll prepare the data for visualization of popular topic distributions across some countries.

In [1]:
from gensim import corpora, models, similarities, matutils
import itertools
import numpy as np
import pandas as pd
from unidecode import unidecode

In [2]:
dropbox = "/Users/WSKIM/Dropbox/moviemeta/"

We can construct a topic matrix from imdb data, as follows:

In [4]:
imdb_meta_df = pd.read_csv(dropbox + 'imdb_meta_df_cleaned.csv')
imdb_topic_df = pd.read_csv(dropbox + 'imdb_topic_df.csv')
imdb_topic_df['title'] = imdb_topic_df['title'].apply(unidecode)
imdb_meta_df['title'] = imdb_meta_df['title'].apply(unidecode)

C:\Anaconda\lib\site-packages\pandas\core\series.py:2169: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [5]:
imdb_meta_df.head()

,Unnamed: 0,Unnamed: 0.1,title,year
0,1,1,#1 Serial Killer (2013),2013
1,4,4,#29 (2012),2012
2,5,5,#30 (2013),2013
3,7,7,#47 (2014),2014
4,8,8,#5 (2013),2013


In [6]:
imdb_topic_df.shape

(199795, 34)

Now we have dataframe of topic matrix for all movies in imdb : column 0 ~29 represents 30 topics, and values in each column indicates the relative prevalence of that topic in each movie. 

In [7]:
imdb_topic_df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,23,24,25,26,27,28,29,Unnamed: 0.1,title,year
0,1,0,0,0,0,0.000000,0.096385,0,0.000000,0,...,0.000000,0.659350,0.000000,0.000000,0.000000,0,0,1,#1 Serial Killer (2013),2013
1,4,0,0,0,0,0.037337,0.000000,0,0.800602,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,4,#29 (2012),2012
2,5,0,0,0,0,0.000000,0.000000,0,0.000000,0,...,0.000000,0.229568,0.000000,0.000000,0.069438,0,0,5,#30 (2013),2013
3,7,0,0,0,0,0.000000,0.291638,0,0.290491,0,...,0.000000,0.000000,0.035633,0.030289,0.000000,0,0,7,#47 (2014),2014
4,8,0,0,0,0,0.000000,0.000000,0,0.516517,0,...,0.223738,0.000000,0.107120,0.000000,0.000000,0,0,8,#5 (2013),2013


In [15]:
imdb_topic_df['title'] = imdb_topic_df['title'].apply(lambda x : x.split('(')[0].strip())

To create dataframes of top movies for each country, let's check the available number of top movie titles and plots data first. 

In [12]:
#Top movies for 8 countries (US, GE, UK, South Korea, Japan, Hong Kong, Russia, Mexico) 
topmovies_df=pd.read_csv(dropbox + "TopMovies_final.csv")

In [13]:
#Top movies for each country
kr_df = topmovies_df[topmovies_df.country=="South Korea"] #available since 2007
jp_df = topmovies_df[topmovies_df.country=="Japan"] #available since 2002
hk_df = topmovies_df[topmovies_df.country=="Hong Kong"] #available since 2002
uk_df = topmovies_df[topmovies_df.country=="United Kingdom"] #available since 2002
ge_df = topmovies_df[topmovies_df.country=="Germany"] #available since 2002
us_df = topmovies_df[topmovies_df.country=="United States"] #available since 2001
ru_df = topmovies_df[topmovies_df.country=="Russia"]
mx_df = topmovies_df[topmovies_df.country=="Mexico"]
kr_df.shape, jp_df.shape, hk_df.shape, uk_df.shape, ge_df.shape, us_df.shape, ru_df.shape, mx_df.shape

((2553, 3),
 (1300, 3),
 (1300, 3),
 (4781, 3),
 (4139, 3),
 (6835, 3),
 (1599, 3),
 (1599, 3))

In [16]:
#collect movies data from imdb_topic_df : check the minimum number of available data, since 2007
years = range(2007, 2015)
available_kr =[]
available_jp =[]
available_hk =[]
available_uk =[]
available_ge =[]
available_us =[]
available_ru =[]
available_mx =[]
for year in years:
    num_movies_kr = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(kr_df[kr_df['year']==year].movie.tolist())]
    available_kr.append(num_movies_kr.shape)
    num_movies_jp = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(jp_df[jp_df['year']==year].movie.tolist())]
    available_jp.append(num_movies_jp.shape)
    num_movies_hk = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(hk_df[hk_df['year']==year].movie.tolist())]
    available_hk.append(num_movies_hk.shape)
    num_movies_uk = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(uk_df[uk_df['year']==year].movie.tolist())]
    available_uk.append(num_movies_uk.shape)
    num_movies_ge = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(ge_df[ge_df['year']==year].movie.tolist())]
    available_ge.append(num_movies_ge.shape)
    num_movies_us = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(us_df[us_df['year']==year].movie.tolist())]
    available_us.append(num_movies_us.shape)
    num_movies_ru = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(ru_df[ru_df['year']==year].movie.tolist())]
    available_ru.append(num_movies_ru.shape)
    num_movies_mx = imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(mx_df[mx_df['year']==year].movie.tolist())]
    available_mx.append(num_movies_mx.shape)
print "Available movies for South Korea : " ,available_kr
print "Available movies for Japan : " ,available_jp
print "Available movies for Hong Kong : " ,available_hk
print "Available movies for United Kingdom : " ,available_uk
print "Available movies for Germany : " ,available_ge
print "Available movies for United States : " ,available_us
print "Available movies for Russia : " ,available_ru
print "Available movies for Mexico : " ,available_mx 

Available movies for South Korea :  [(72, 34), (67, 34), (62, 34), (63, 34), (60, 34), (49, 34), (50, 34), (50, 34)]
Available movies for Japan :  [(28, 34), (30, 34), (21, 34), (30, 34), (22, 34), (21, 34), (23, 34), (27, 34)]
Available movies for Hong Kong :  [(44, 34), (42, 34), (43, 34), (41, 34), (41, 34), (40, 34), (42, 34), (56, 34)]
Available movies for United Kingdom :  [(181, 34), (184, 34), (135, 34), (127, 34), (178, 34), (182, 34), (166, 34), (158, 34)]
Available movies for Germany :  [(121, 34), (100, 34), (101, 34), (92, 34), (98, 34), (109, 34), (101, 34), (117, 34)]
Available movies for United States :  [(216, 34), (214, 34), (198, 34), (189, 34), (183, 34), (179, 34), (172, 34), (163, 34)]
Available movies for Russia :  [(93, 34), (90, 34), (89, 34), (94, 34), (91, 34), (88, 34), (83, 34), (81, 34)]
Available movies for Mexico :  [(88, 34), (83, 34), (72, 34), (77, 34), (77, 34), (80, 34), (73, 34), (71, 34)]


In [28]:
imdb_topic_df[imdb_topic_df['year']==year][imdb_topic_df['title'].isin(us_df[us_df['year']==year].movie.tolist())]

(11, 34)

As we can see from above results, data for some Asian countires are not enough. Thus, it would be better to limit our analysis on topic distribution to Russia, South Korea, United States, Germany, United Kingdom, and Mexico : we can use 50 movies for each year since 2007, because the number of movies should be same for all countries to see if there is any similarity or difference in movie topics distribution for those countries. It's very small number of movies, but let's explore the data to see if tehre are differences across those countries.

In [17]:
def get(df, year, country=None):
    df = df[df['year']==year]
    if country:
        return df[df['country']==country]
    return df

imdb_merged_df = pd.DataFrame()

for year in range(2001, 2015):
    for country in ['Germany','United States', 'United Kingdom', 'South Korea', 'Russia', 'Mexico']:
        topic = get(imdb_topic_df, year)
        top = get(topmovies_df, year,country)
        #remove date from imdb title
        titles = topic.title.apply(lambda x : x.split('(')[0].strip().lower())
        topic = topic[titles.isin([t.lower() for t in top.movie.tolist()])]
        topic['country'] = country
        imdb_merged_df = pd.concat([imdb_merged_df, topic])
imdb_merged_df.head()    

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,24,25,26,27,28,29,Unnamed: 0.1,title,year,country
4318,5579,0.093824,0.115534,0.107758,0.055557,0.000000,0.126320,0.076326,0.000000,0.000000,...,0.000000,0,0.000000,0.055111,0.000000,0.000000,5579,A Knight's Tale,2001,Germany
10792,13945,0.012616,0.065517,0.000000,0.000000,0.023750,0.000000,0.000000,0.015162,0.044170,...,0.251304,0,0.000000,0.000000,0.000000,0.401107,13945,Along Came a Spider,2001,Germany
11278,14601,0.000000,0.012200,0.000000,0.000000,0.016742,0.034204,0.017494,0.000000,0.047177,...,0.000000,0,0.000000,0.000000,0.212286,0.095194,14601,America's Sweethearts,2001,Germany
11518,14900,0.020518,0.082306,0.000000,0.102754,0.000000,0.106369,0.000000,0.000000,0.000000,...,0.000000,0,0.021140,0.000000,0.210141,0.032807,14900,American Pie 2,2001,Germany
12869,16626,0.000000,0.000000,0.000000,0.000000,0.000000,0.283570,0.000000,0.000000,0.000000,...,0.156316,0,0.011424,0.000000,0.049496,0.132076,16626,Angel Eyes,2001,Germany


In [18]:
#filtering all movies available in imdb for each country
imdb_df_us_all = imdb_merged_df[imdb_merged_df['country']=='United States']
imdb_df_ge_all = imdb_merged_df[imdb_merged_df['country']=='Germany']
imdb_df_uk_all = imdb_merged_df[imdb_merged_df['country']=='United Kingdom']
imdb_df_kr_all = imdb_merged_df[imdb_merged_df['country']=='South Korea']
imdb_df_ru_all = imdb_merged_df[imdb_merged_df['country']=='Russia']
imdb_df_mx_all = imdb_merged_df[imdb_merged_df['country']=='Mexico']
imdb_df_us_all.shape, imdb_df_ge_all.shape, imdb_df_uk_all.shape, imdb_df_kr_all.shape, imdb_df_ru_all.shape, imdb_df_mx_all.shape

((2785, 35), (1425, 35), (1997, 35), (488, 35), (731, 35), (648, 35))

In [30]:
us_df.head()

,country,year,movie
4139,United States,2014,American Sniper
4140,United States,2014,The Hunger Games: Mockingjay - Part 1
4141,United States,2014,Guardians of the Galaxy
4142,United States,2014,Captain America: The Winter Soldier
4143,United States,2014,The LEGO Movie


In [32]:
#rename column in us_df,etc.
us_df=us_df.rename(columns = {'movie':'title'})
ge_df=ge_df.rename(columns = {'movie':'title'})
uk_df=uk_df.rename(columns = {'movie':'title'})
kr_df=kr_df.rename(columns = {'movie':'title'})
ru_df=ru_df.rename(columns = {'movie':'title'})   
mx_df=mx_df.rename(columns = {'movie':'title'})

In [34]:
#merge with top movies dataframe(ex. us_df) with imdb_df_us_all, using 'title' as key to maintain the rank structure
imdb_us_top = pd.merge(us_df, imdb_df_us_all, on='title')
imdb_ge_top = pd.merge(ge_df, imdb_df_ge_all, on='title')
imdb_uk_top = pd.merge(uk_df, imdb_df_uk_all, on='title')
imdb_kr_top = pd.merge(kr_df, imdb_df_kr_all, on='title')
imdb_ru_top = pd.merge(ru_df, imdb_df_ru_all, on='title')
imdb_mx_top = pd.merge(mx_df, imdb_df_mx_all, on='title')
imdb_us_top.head()

,country_x,year_x,title,Unnamed: 0,0,1,2,3,4,5,...,23,24,25,26,27,28,29,Unnamed: 0.1,year_y,country_y
0,United States,2014,American Sniper,14965,0.000000,0.043908,0,0.056497,0.000000,0.000000,...,0.029536,0.000000,0.000000,0.027191,0.000000,0.188507,0.018732,14965,2014,United States
1,United States,2014,The Hunger Games: Mockingjay - Part 1,212123,0.000000,0.079328,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.072179,0.000000,0.104211,212123,2014,United States
2,United States,2014,Guardians of the Galaxy,84036,0.000000,0.000000,0,0.012578,0.000000,0.000000,...,0.000000,0.147887,0.000000,0.028995,0.141668,0.032615,0.126039,84036,2014,United States
3,United States,2014,Captain America: The Winter Soldier,37348,0.000000,0.011638,0,0.000000,0.017578,0.076597,...,0.000000,0.160758,0.017021,0.000000,0.011759,0.052107,0.030686,37348,2014,United States
4,United States,2014,The Hobbit: The Battle of the Five Armies,211585,0.039721,0.000000,0,0.125411,0.000000,0.027596,...,0.041084,0.027410,0.018114,0.017507,0.263708,0.011811,0.040026,211585,2014,United States


In [38]:
#drop duplicated columns
imdb_us_top = imdb_us_top.drop(imdb_us_top.columns[[35,36]], axis=1)
imdb_ge_top = imdb_ge_top.drop(imdb_ge_top.columns[[35,36]], axis=1)
imdb_uk_top = imdb_uk_top.drop(imdb_uk_top.columns[[35,36]], axis=1)
imdb_kr_top = imdb_kr_top.drop(imdb_kr_top.columns[[35,36]], axis=1)   
imdb_ru_top = imdb_ru_top.drop(imdb_ru_top.columns[[35,36]], axis=1)
imdb_mx_top = imdb_mx_top.drop(imdb_mx_top.columns[[35,36]], axis=1)
imdb_us_top.head(3)

,country_x,year_x,title,Unnamed: 0,0,1,2,3,4,5,...,21,22,23,24,25,26,27,28,29,Unnamed: 0.1
0,United States,2014,American Sniper,14965,0,0.043908,0,0.056497,0,0,...,0.113393,0.039953,0.029536,0.000000,0,0.027191,0.000000,0.188507,0.018732,14965
1,United States,2014,The Hunger Games: Mockingjay - Part 1,212123,0,0.079328,0,0.000000,0,0,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.072179,0.000000,0.104211,212123
2,United States,2014,Guardians of the Galaxy,84036,0,0.000000,0,0.012578,0,0,...,0.073377,0.000000,0.000000,0.147887,0,0.028995,0.141668,0.032615,0.126039,84036


Now we have dataframes containing topics for each movie, in each country, maintaining the rank structure. To select top 50(first 50) movies for each year, drop null rows first.

In [40]:
#number of rows in each dataframe before dropping null rows
imdb_us_top.shape[0], imdb_ge_top.shape[0],imdb_uk_top.shape[0], imdb_kr_top.shape[0], imdb_ru_top.shape[0], imdb_mx_top.shape[0], 

(2710, 1384, 1955, 473, 709, 621)

In [49]:
#check if there are missing data in the dataframes
#imdb_us_top.count(axis=0) #2710 - no missing
#imdb_ge_top.count(axis=0) #1384 - no missing
#imdb_uk_top.count(axis=0) #1955 - no missing
#imdb_kr_top.count(axis=0) #473 - no missing
#imdb_ru_top.count(axis=0) #709 - no missing
#imdb_mx_top.count(axis=0) #621 - no missing

In [58]:
#selecting first 50 movies for each year in each country
imdb_us_top50_df = pd.DataFrame()
imdb_ge_top50_df = pd.DataFrame()
imdb_uk_top50_df = pd.DataFrame()
imdb_kr_top50_df = pd.DataFrame()
imdb_ru_top50_df = pd.DataFrame()
imdb_mx_top50_df = pd.DataFrame()
years = range(2007, 2015)
for year in years:
    us_yearly_tops = imdb_us_top[imdb_us_top['year_x']==year].head(50)
    ge_yearly_tops = imdb_ge_top[imdb_ge_top['year_x']==year].head(50)
    uk_yearly_tops = imdb_uk_top[imdb_uk_top['year_x']==year].head(50)
    kr_yearly_tops = imdb_kr_top[imdb_kr_top['year_x']==year].head(50)
    ru_yearly_tops = imdb_ru_top[imdb_ru_top['year_x']==year].head(50)
    mx_yearly_tops = imdb_mx_top[imdb_mx_top['year_x']==year].head(50)
    imdb_us_top50_df = pd.concat([imdb_us_top50_df, us_yearly_tops])
    imdb_ge_top50_df = pd.concat([imdb_ge_top50_df, ge_yearly_tops]) 
    imdb_uk_top50_df = pd.concat([imdb_uk_top50_df, uk_yearly_tops]) 
    imdb_kr_top50_df = pd.concat([imdb_kr_top50_df, kr_yearly_tops])
    imdb_ru_top50_df = pd.concat([imdb_ru_top50_df, ru_yearly_tops]) 
    imdb_mx_top50_df = pd.concat([imdb_mx_top50_df, mx_yearly_tops]) 
imdb_us_top50_df.shape[0], imdb_ge_top50_df.shape[0], imdb_uk_top50_df.shape[0], imdb_kr_top50_df.shape[0], imdb_ru_top50_df.shape[0], imdb_mx_top50_df.shape[0]

(400, 400, 400, 399, 400, 400)

Finally, we obtained dataframe containing 400 movie plots for each country. 

In [59]:
#save 
imdb_us_top50_df.to_csv("imdb_us_top50_df.csv", index=False)
imdb_ge_top50_df.to_csv("imdb_ge_top50_df.csv", index=False)
imdb_uk_top50_df.to_csv("imdb_uk_top50_df.csv", index=False)
imdb_kr_top50_df.to_csv("imdb_kr_top50_df.csv", index=False)
imdb_ru_top50_df.to_csv("imdb_ru_top50_df.csv", index=False)
imdb_mx_top50_df.to_csv("imdb_mx_top50_df.csv", index=False)

In [60]:
imdb_us_top50_df.head(3)

,country_x,year_x,title,Unnamed: 0,0,1,2,3,4,5,...,21,22,23,24,25,26,27,28,29,Unnamed: 0.1
1298,United States,2007,Spider-Man 3,187359,0.000000,0.020595,0.000000,0.028429,0.000000,0.103724,...,0.156983,0,0.061324,0.100964,0,0.012752,0.070043,0.084983,0.000000,187359
1299,United States,2007,Spider-Man 3,187360,0.017332,0.035922,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.034605,0.124082,0,0.000000,0.064252,0.000000,0.170678,187360
1300,United States,2007,Shrek the Third,180798,0.000000,0.000000,0.099467,0.000000,0.036665,0.000000,...,0.052788,0,0.025903,0.000000,0,0.024062,0.219669,0.306300,0.000000,180798


In [64]:
#topic scores for top movies for each country
kr_topic = imdb_kr_top50_df[imdb_kr_top50_df.columns[4:34]] #score for each topic is in column 0~29
kr_topic_score= kr_topic.sum(axis=0)
us_topic = imdb_us_top50_df[imdb_us_top50_df.columns[4:34]]
us_topic_score= us_topic.sum(axis=0)
uk_topic = imdb_uk_top50_df[imdb_uk_top50_df.columns[4:34]]
uk_topic_score= uk_topic.sum(axis=0)
ge_topic = imdb_ge_top50_df[imdb_ge_top50_df.columns[4:34]]
ge_topic_score= ge_topic.sum(axis=0)
ru_topic = imdb_ru_top50_df[imdb_ru_top50_df.columns[4:34]]
ru_topic_score= ru_topic.sum(axis=0)
mx_topic = imdb_mx_top50_df[imdb_mx_top50_df.columns[4:34]]
mx_topic_score= mx_topic.sum(axis=0)
len(kr_topic_score), len(us_topic_score), len(uk_topic_score), len(ge_topic_score), len(ru_topic_score), len(mx_topic_score)

(30, 30, 30, 30, 30, 30)

In [65]:
#Create dataframe for visualization (since 2007, full year)
country_us = ['United States'] *30
country_ge = ['Germany']*30
country_uk = ['United Kingdom']*30
country_kr = ['South Korea'] *30
country_ru = ['Russia']*30
country_mx = ['Mexico']*30
topics =['magic, myths', 'school, college', 'fantasy, christmas', 'home', 'ships, sailing, pirates','love, relationships',
         'war','exploration, nature, space', 'comedy', 'places, nature, scenery', 'hollywood, stars', 'society, culture',
         'historical, costumes', 'money, robbery','photography, design','spies, terrorism', 'mixed', 'town', 'press, politics',
         'crime, police, underworld','documentary, interview','friendship, relationships','cowboys and indians', 'night life, enjoyment',
         'crime, mistery','music', 'farming, country side', 'fantasy, fairy tale','love, family', 'gangs, drugs, police']
data_us = {'Country':country_us, 'Topic':topics, 'Topic_Score':us_topic_score}
data_ge = {'Country':country_ge, 'Topic':topics, 'Topic_Score':ge_topic_score}
data_uk = {'Country':country_uk, 'Topic':topics, 'Topic_Score':uk_topic_score}
data_kr = {'Country':country_kr, 'Topic':topics, 'Topic_Score':kr_topic_score} 
data_ru = {'Country':country_ru, 'Topic':topics, 'Topic_Score':ru_topic_score}
data_mx = {'Country':country_mx, 'Topic':topics, 'Topic_Score':mx_topic_score}

us = pd.DataFrame(data_us)
ge = pd.DataFrame(data_ge)
uk = pd.DataFrame(data_uk)
kr = pd.DataFrame(data_kr)
ru = pd.DataFrame(data_ru)
mx = pd.DataFrame(data_mx)

frames =[us,ge, uk, kr, ru, mx]
popular_df= pd.concat(frames)

In [66]:
popular_df.head()

,Country,Topic,Topic_Score
0,United States,"magic, myths",3.242855
1,United States,"school, college",16.165176
2,United States,"fantasy, christmas",3.716660
3,United States,home,9.271523
4,United States,"ships, sailing, pirates",3.583652


In [67]:
#Global top 10 topics for popular movies 
topics =['magic, myths', 'school, college', 'fantasy, christmas', 'home', 'ships, sailing, pirates','love, relationships',
         'war','exploration, nature, space', 'comedy', 'places, nature, scenery', 'hollywood, stars', 'society, culture',
         'historical, costumes', 'money, robbery','photography, design','spies, terrorism', 'mixed', 'town', 'press, politics',
         'crime, police, underworld','documentary, interview','friendship, relationships','cowboys and indians', 'night life, enjoyment',
         'crime, mistery','music', 'farming, country side', 'fantasy, fairy tale','love, family', 'gangs, drugs, police']
from operator import add
us_ge = map(add, us_topic_score, ge_topic_score)
uk_kr = map(add, uk_topic_score, kr_topic_score)
ru_mx = map(add, ru_topic_score, mx_topic_score)
temp_= map(add, us_ge, uk_kr)
global_topic_score = map(add, temp_, ru_mx)
Global = ['Global']*30
global_topic_df = pd.DataFrame({'Country': Global, 'Topic':topics, 'Topic_Score':global_topic_score})
global_topic_df.head(5)

,Country,Topic,Topic_Score
0,Global,"magic, myths",21.014179
1,Global,"school, college",84.374142
2,Global,"fantasy, christmas",23.275571
3,Global,home,54.632174
4,Global,"ships, sailing, pirates",24.593461


In [68]:
#selecting global top 10 topics
global_top = global_topic_df.sort_values('Topic_Score', axis=0, ascending=False, inplace=False)
global_top.head(10) #topic 28, 21, 24, 29, 27, 7, 5, 6, 13, 1

,Country,Topic,Topic_Score
28,Global,"love, family",235.173986
21,Global,"friendship, relationships",217.848011
24,Global,"crime, mistery",192.753606
29,Global,"gangs, drugs, police",174.383837
27,Global,"fantasy, fairy tale",148.605202
7,Global,"exploration, nature, space",144.222312
5,Global,"love, relationships",142.127684
6,Global,war,136.103304
13,Global,"money, robbery",106.966953
1,Global,"school, college",84.374142


In [69]:
#filtering global top10 topics for each country 
global_topics = ['love, relationships', 'exploration, nature, space','love, family', 'friendship, relationships', 'society, culture',
                 'documentary, interview', 'crime, mistery','gangs, drugs, police','money, robbery','home']
countrywise_global_topic = popular_df[popular_df['Topic'].isin(global_topics)]
countrywise_global_topic.head()

,Country,Topic,Topic_Score
3,United States,home,9.271523
5,United States,"love, relationships",24.419085
7,United States,"exploration, nature, space",23.412903
11,United States,"society, culture",9.284826
13,United States,"money, robbery",18.458428


In [70]:
#save data as csv for visualization
popularity = pd.concat([global_topic_df, popular_df])
popularity.to_csv("popularity_new.csv", index= False)

In [71]:
popularity.head()

,Country,Topic,Topic_Score
0,Global,"magic, myths",21.014179
1,Global,"school, college",84.374142
2,Global,"fantasy, christmas",23.275571
3,Global,home,54.632174
4,Global,"ships, sailing, pirates",24.593461


Let's get prevalent terms in each topic, to make our visualization more meaningful and understandable to our audience. We can the topic-terms distribution using lda.get_topic_terms(topic id). 

In [73]:
#load ldamodel, corpus, and dictinary
imdb_lda = models.LdaModel.load(dropbox +'lda_imdb.model')
imdb_corpus = corpora.MmCorpus(dropbox +'lda_imdb.corpus')
imdb_dict = corpora.Dictionary.load(dropbox +'lda_imdb.dict')

In [74]:
#prevalent terms for each topics
ids = range(0, 30)
topic_terms =[]
for id in ids:
    terms = imdb_lda.get_topic_terms(id,topn=5) #[(id, score), (id, score),...]
    topic_terms.append([x[0] for x in terms])
topic_terms[:3] #collection of top 5 prevalent terms in each topic

[[34322, 1240, 30396, 10059, 19977],
 [7575, 30745, 10710, 26115, 10567],
 [25939, 17413, 13018, 23922, 13016]]

In [75]:
#convert id to word
words = []
for topic in topic_terms:
    words.append(map(lambda id:imdb_dict[id], topic))
words

[[u'brown', u'yoga', u'sir', u'prize', u'disabl'],
 [u'school', u'student', u'team', u'game', u'colleg'],
 [u'dog', u'la', u'de', u'christma', u'di'],
 [u'enter', u'home', u'scene', u'leav', u'arriv'],
 [u'ship', u'danc', u'dancer', u'crew', u'captain'],
 [u'love', u'past', u'woman', u'relationship', u'friend'],
 [u'war', u'armi', u'soldier', u'fight', u'forc'],
 [u'human', u'vs', u'explor', u'experi', u'creat'],
 [u'adventur', u'girl', u'episod', u'comedi', u'comic'],
 [u'island', u'villag', u'water', u'sea', u'river'],
 [u'star', u'book', u'titl', u'movi', u'writer'],
 [u'american', u'famili', u'commun', u'documentari', u'cultur'],
 [u'tree', u'dress', u'servant', u'tabl', u'wear'],
 [u'money', u'plan', u'girl', u'tri', u'escap'],
 [u'pictur', u'anim', u'imag', u'color', u'view'],
 [u'american', u'japanes', u'russian', u'govern', u'pilot'],
 [u'lucki', u'zero', u'dive', u'camp', u'tag'],
 [u'town', u'local', u'disappoint', u'station', u'phone'],
 [u'parti', u'smith', u'newspap', u'ho

In [76]:
words_ascii=[]
for topic in words:
    words_ascii.append(map(lambda term:term.encode('ascii'), topic))
words_ascii

[['brown', 'yoga', 'sir', 'prize', 'disabl'],
 ['school', 'student', 'team', 'game', 'colleg'],
 ['dog', 'la', 'de', 'christma', 'di'],
 ['enter', 'home', 'scene', 'leav', 'arriv'],
 ['ship', 'danc', 'dancer', 'crew', 'captain'],
 ['love', 'past', 'woman', 'relationship', 'friend'],
 ['war', 'armi', 'soldier', 'fight', 'forc'],
 ['human', 'vs', 'explor', 'experi', 'creat'],
 ['adventur', 'girl', 'episod', 'comedi', 'comic'],
 ['island', 'villag', 'water', 'sea', 'river'],
 ['star', 'book', 'titl', 'movi', 'writer'],
 ['american', 'famili', 'commun', 'documentari', 'cultur'],
 ['tree', 'dress', 'servant', 'tabl', 'wear'],
 ['money', 'plan', 'girl', 'tri', 'escap'],
 ['pictur', 'anim', 'imag', 'color', 'view'],
 ['american', 'japanes', 'russian', 'govern', 'pilot'],
 ['lucki', 'zero', 'dive', 'camp', 'tag'],
 ['town', 'local', 'disappoint', 'station', 'phone'],
 ['parti', 'smith', 'newspap', 'hotel', 'elect'],
 ['brother', 'inspector', 'kill', 'twin', 'polic'],
 ['interview', 'documentar

In [77]:
#add column 'words' in the dataframe 'popularity'
popularity['words']=pd.Series(words_ascii*7, index=popularity.index)
popularity.head()

,Country,Topic,Topic_Score,words
0,Global,"magic, myths",21.014179,"[brown, yoga, sir, prize, disabl]"
1,Global,"school, college",84.374142,"[school, student, team, game, colleg]"
2,Global,"fantasy, christmas",23.275571,"[dog, la, de, christma, di]"
3,Global,home,54.632174,"[enter, home, scene, leav, arriv]"
4,Global,"ships, sailing, pirates",24.593461,"[ship, danc, dancer, crew, captain]"


In [78]:
popularity.tail()

,Country,Topic,Topic_Score,words
25,Mexico,music,5.062026,"[music, artist, perform, band, song]"
26,Mexico,"farming, country side",3.154734,"[farmer, toy, bride, farm, hat]"
27,Mexico,"fantasy, fairy tale",25.881581,"[mountain, king, evil, zombi, power]"
28,Mexico,"love, family",43.444800,"[love, marri, wife, father, famili]"
29,Mexico,"gangs, drugs, police",28.638080,"[polic, gang, prison, drug, crime]"


In [79]:
#save final 'popularity' to csv
popularity.to_csv("popularity_new.csv", index= False)

Note : 'popularity_new.csv' contains global data & data for 6 countries, with prevalent 5 words for each topic. 

## Topic Distribution with Origin of Popular Movies

Are there any relationship between topic distribution and origin of movies? In other words, do the movies made in United States show different topic distribution from those made in South Korea? To answer this question, we are going to investigate the topic distribution of popular movies with origin of movies. For that purpose, create a dataframe "imdb_meta_top_df" containing origin of movies for popular movies first. 

In [369]:
#filtering the topic matrix by popular movies
imdb_topic_top_df = pd.DataFrame()

for year in range(2001, 2015):
    for country in ['Germany','United States', 'United Kingdom', 'South Korea', 'Japan', 'Hong Kong']:
        topic = get(imdb_topic_df, year)
        top = get(topmovies_df, year,country)
        #remove date from imdb title, compare lower cased titles
        titles = topic.title.apply(lambda x : x.split('(')[0].strip().lower())
        topic = topic[titles.isin([t.lower() for t in top.movie.tolist()])]
        topic['country'] = country
        imdb_topic_top_df = pd.concat([imdb_topic_top_df, topic])
imdb_topic_top_df.head()  

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,Unnamed: 0,title,year,country
5579,0.000000,0,0.000000,0.250719,0.000000,0.000000,0.040608,0.000000,0.000000,0.05785,...,0.000000,0.081993,0.084435,0.074651,0.000000,0.039129,5579,A Knight's Tale (2001),2001,Germany
13945,0.000000,0,0.029535,0.000000,0.000000,0.151568,0.000000,0.070572,0.000000,0.00000,...,0.149090,0.000000,0.000000,0.000000,0.046381,0.000000,13945,Along Came a Spider (2001),2001,Germany
14601,0.000000,0,0.027998,0.000000,0.090975,0.000000,0.130275,0.000000,0.027781,0.00000,...,0.070602,0.000000,0.015746,0.061785,0.000000,0.173716,14601,America's Sweethearts (2001),2001,Germany
14900,0.000000,0,0.000000,0.000000,0.000000,0.106993,0.000000,0.663135,0.000000,0.00000,...,0.000000,0.062978,0.038708,0.000000,0.000000,0.097321,14900,American Pie 2 (2001),2001,Germany
16626,0.029772,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.030694,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.387287,0.000000,16626,Angel Eyes (2001),2001,Germany


In [380]:
#merge two dataframes imdb_topic_top_df and imdb_meta_top_df
imdb_top_origin_df = pd.merge(imdb_topic_top_df, imdb_meta_top_df, on='title')
imdb_top_origin_df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,Unnamed: 0_x,title,year_x,country,Unnamed: 0_y,Unnamed: 0.1,year_y,origin
0,0,0,0.000000,0.250719,0.000000,0.000000,0.040608,0.000000,0.000000,0.05785,...,0.000000,0.039129,5579,A Knight's Tale (2001),2001,Germany,5579,5579,2001,USA
1,0,0,0.000000,0.250719,0.000000,0.000000,0.040608,0.000000,0.000000,0.05785,...,0.000000,0.039129,5579,A Knight's Tale (2001),2001,United States,5579,5579,2001,USA
2,0,0,0.029535,0.000000,0.000000,0.151568,0.000000,0.070572,0.000000,0.00000,...,0.046381,0.000000,13945,Along Came a Spider (2001),2001,Germany,13945,13945,2001,Canada
3,0,0,0.029535,0.000000,0.000000,0.151568,0.000000,0.070572,0.000000,0.00000,...,0.046381,0.000000,13945,Along Came a Spider (2001),2001,United States,13945,13945,2001,Canada
4,0,0,0.027998,0.000000,0.090975,0.000000,0.130275,0.000000,0.027781,0.00000,...,0.000000,0.173716,14601,America's Sweethearts (2001),2001,Germany,14601,14601,2001,USA


In [383]:
#check the number 
origins=imdb_top_origin_df['origin'].tolist() #len(origins) = 3761 
origins = list(set(origins))  #after removing duplicate, len(origins) = 77
origins[:6]

['Canada', 'Turkey', 'Italy', 'Czech Republic', 'USA', 'Afghanistan']

In [385]:
grouped = imdb_top_origin_df.groupby('origin')

In [394]:
origin_topic_sum= grouped.sum()
origin_topic_sum.head(5)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,Unnamed: 0_x,year_x,Unnamed: 0_y,Unnamed: 0.1,year_y
origin,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.000000,0.055250,0.000000,0.000000,0.040695,0.000000,0.209245,0.000000,0.000000,0.000000,...,0.065245,0.000000,0.000000,0.069425,0.179746,88325,2011,88325,88325,2011
Argentina,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.529603,0.000000,0.000000,0.000000,...,0.455610,0.000000,0.000000,0.250142,0.000000,674015,8040,674015,674015,8040
Australia,0.739340,4.774235,1.647710,3.457284,1.363858,16.394829,5.036436,10.928599,2.202809,1.122519,...,7.545632,1.406462,4.682922,13.168547,7.206612,27140833,413226,27140833,27140833,413226
Austria,0.112778,0.151980,0.098123,0.723472,0.072395,2.051567,0.441433,0.825162,0.511871,0.237841,...,0.134382,0.013983,0.698932,1.191857,0.195471,650029,24113,650029,650029,24113
Bahamas,0.000000,0.261953,0.000000,0.000000,0.000000,0.492779,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.495543,0.412132,0.000000,191080,10030,191080,191080,10030


In [433]:
#subdataframe containing only topcis score
origin_topic_score = origin_topic_sum.iloc[:, :30]
origin_topic_score.head(5)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
origin,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.000000,0.055250,0.000000,0.000000,0.040695,0.000000,0.209245,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.058815,0.065245,0.000000,0.000000,0.069425,0.179746
Argentina,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.529603,0.000000,0.000000,0.000000,...,0.000000,0.632386,0.000000,0.116809,0.423718,0.455610,0.000000,0.000000,0.250142,0.000000
Australia,0.739340,4.774235,1.647710,3.457284,1.363858,16.394829,5.036436,10.928599,2.202809,1.122519,...,7.717166,17.888649,2.815919,1.791592,10.954605,7.545632,1.406462,4.682922,13.168547,7.206612
Austria,0.112778,0.151980,0.098123,0.723472,0.072395,2.051567,0.441433,0.825162,0.511871,0.237841,...,0.209570,0.365508,0.160856,0.586655,1.274669,0.134382,0.013983,0.698932,1.191857,0.195471
Bahamas,0.000000,0.261953,0.000000,0.000000,0.000000,0.492779,0.000000,0.000000,0.000000,0.000000,...,0.526696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.495543,0.412132,0.000000


In [398]:
#Number of movies in each of origins
num_movies = grouped.count()
num_movies.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,Unnamed: 0_x,title,year_x,country,Unnamed: 0_y,Unnamed: 0.1,year_y
origin,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Argentina,3,3,3,3,3,3,3,3,3,3,...,3,3,3,4,4,4,4,4,4,4
Australia,162,162,162,162,162,162,162,162,162,162,...,162,162,162,206,206,206,206,206,206,206
Austria,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
Bahamas,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [410]:
#to confirm the number in column(1~29) means total number of movies in that origin
imdb_top_origin_df[imdb_top_origin_df.origin =='Afghanistan']

,0,1,2,3,4,5,6,7,8,9,...,28,29,Unnamed: 0_x,title,year_x,country,Unnamed: 0_y,Unnamed: 0.1,year_y,origin
5818,0,0.05525,0,0,0.040695,0,0.209245,0,0,0,...,0.069425,0.179746,88325,Hell and Back Again (2011),2011,United States,88325,88325,2011,Afghanistan


In [412]:
#number in each column(0~29) is equal to total number of movies in that origin(excluding movies without plots)
imdb_top_origin_df[imdb_top_origin_df.origin =='Argentina']

,0,1,2,3,4,5,6,7,8,9,...,28,29,Unnamed: 0_x,title,year_x,country,Unnamed: 0_y,Unnamed: 0.1,year_y,origin
1713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,227792,The Take (2004),2004,United States,227792,227792,2004,Argentina
6167,0,0,0,0,0,0,0.176534,0,0,0,...,0.083381,0,148741,On the Road (2012),2012,Germany,148741,148741,2012,Argentina
6168,0,0,0,0,0,0,0.176534,0,0,0,...,0.083381,0,148741,On the Road (2012),2012,United States,148741,148741,2012,Argentina
6169,0,0,0,0,0,0,0.176534,0,0,0,...,0.083381,0,148741,On the Road (2012),2012,United Kingdom,148741,148741,2012,Argentina


In [511]:
#make dataframe for visualization
n_movies = num_movies[29].tolist()
n_movies = [x for x in n_movies if x != 0]
len(n_movies)

68

In [520]:
from itertools import repeat
n_movie=[x for item in n_movies for x in repeat(item, 30)]

In [415]:
origin_topic_score.shape[0]

76

In [435]:
#drop origins with no available data(rows with NaN due to the absence of movie plots data)
origin_topic_score = origin_topic_score.dropna()
origin_topic_score.shape # 8 origins droppped

(68, 30)

In [436]:
origin_topic_score = origin_topic_score.reset_index()
origin_topic_score.head()

,origin,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,Afghanistan,0.000000,0.055250,0.000000,0.000000,0.040695,0.000000,0.209245,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.058815,0.065245,0.000000,0.000000,0.069425,0.179746
1,Argentina,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.529603,0.000000,0.000000,...,0.000000,0.632386,0.000000,0.116809,0.423718,0.455610,0.000000,0.000000,0.250142,0.000000
2,Australia,0.739340,4.774235,1.647710,3.457284,1.363858,16.394829,5.036436,10.928599,2.202809,...,7.717166,17.888649,2.815919,1.791592,10.954605,7.545632,1.406462,4.682922,13.168547,7.206612
3,Austria,0.112778,0.151980,0.098123,0.723472,0.072395,2.051567,0.441433,0.825162,0.511871,...,0.209570,0.365508,0.160856,0.586655,1.274669,0.134382,0.013983,0.698932,1.191857,0.195471
4,Bahamas,0.000000,0.261953,0.000000,0.000000,0.000000,0.492779,0.000000,0.000000,0.000000,...,0.526696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.495543,0.412132,0.000000


Let's make a dataframe for visualization using Tableau public, containing origin, topic label, topic score, and # of movies.

In [455]:
new_origin = origin_topic_score['origin'].tolist()
len(new_origin) #list of origins

68

In [473]:
origin_list=[]
for origin in new_origin:
    origin_list.append([origin]*30)
len(origin_list)

68

In [495]:
topic_list = ['magic, myths', 'school, college', 'fantasy, christmas', 'home', 'ships, sailing, pirates', 'love, relationships', 'war', 'exploration, nature, space', 
              'comedy','places, nature, scenery','hollywood, stars', 'society, culture','historical, costumes', 'money, robbery',
              'photography, design','spies, terrorism', 'mixed','town','press, politics', 'crime, police, underworld',
             'documentary, interview', 'friendship, relationships', 'cowboys and indians','night life, enjoyment', 'crime, mistery',  
              'music', 'farming, country side','fantasy, fairy tale', 'love, family', 'gangs, drugs, police']
topics = [topic_list]*68

In [462]:
df = origin_topic_score.iloc[:, 1:31]
df.head()
t = map(list, df.values) #list of topic scores for each origin
len(t)

68

In [507]:
#make dataframe for visualization on Tableau public
origin_df =pd.DataFrame()
num_origin= range(0,68)
for i in num_origin:
    temp = pd.DataFrame({'origin':origin_list[i], 'topic': topics[i] , 'topic_score': t[i]})
    origin_df = pd.concat([origin_df, temp])

    origin_df.tail()                       

,origin,topic,topic_score
25,Vietnam,music,0.109885
26,Vietnam,"farming, country side",0.000000
27,Vietnam,"fantasy, fairy tale",0.000000
28,Vietnam,"love, family",0.000000
29,Vietnam,"gangs, drugs, police",0.000000


In [546]:
#add number of movies as a column
origin_df['num_movies'] = pd.Series(n_movie, index=origin_df.index)
origin_df.tail()

,origin,topic,topic_score,num_movies,normalized_score
25,Vietnam,music,0.109885,1,3.296556
26,Vietnam,"farming, country side",0.000000,1,0.000000
27,Vietnam,"fantasy, fairy tale",0.000000,1,0.000000
28,Vietnam,"love, family",0.000000,1,0.000000
29,Vietnam,"gangs, drugs, police",0.000000,1,0.000000


In [548]:
from operator import truediv
s = origin_df['topic_score'].tolist()
n = origin_df['num_movies'].tolist()
n_score = map(truediv, s, n)
origin_df['normalized_score'] = pd.Series(n_score, index=origin_df.index)
origin_df.head()

,origin,topic,topic_score,num_movies,normalized_score
0,Afghanistan,"magic, myths",0.000000,1,0.000000
1,Afghanistan,"school, college",0.055250,1,0.055250
2,Afghanistan,"fantasy, christmas",0.000000,1,0.000000
3,Afghanistan,home,0.000000,1,0.000000
4,Afghanistan,"ships, sailing, pirates",0.040695,1,0.040695


The num_mobies should be divided by number of topics(30), because Tableau public plots the sum of num_mobies for each origin.

In [549]:
origin_df['num_movies'] = origin_df['num_movies']/30
origin_df.head()

,origin,topic,topic_score,num_movies,normalized_score
0,Afghanistan,"magic, myths",0.000000,0.033333,0.000000
1,Afghanistan,"school, college",0.055250,0.033333,0.055250
2,Afghanistan,"fantasy, christmas",0.000000,0.033333,0.000000
3,Afghanistan,home,0.000000,0.033333,0.000000
4,Afghanistan,"ships, sailing, pirates",0.040695,0.033333,0.040695


In [551]:
origin_df = origin_df.sort_values('num_movies', axis=0, ascending=False, inplace=False)
origin_df.head()

,origin,topic,topic_score,num_movies,normalized_score
15,USA,"spies, terrorism",63.330414,106.4,0.019840
16,USA,mixed,29.020412,106.4,0.009092
2,USA,"fantasy, christmas",29.022633,106.4,0.009092
3,USA,home,119.264356,106.4,0.037364
4,USA,"ships, sailing, pirates",32.048349,106.4,0.010040


In [552]:
#save to csv 
origin_df.to_csv("origin_df.csv", index=False)

In [532]:
#origins with number of movies greater than 50
num_df = pd.DataFrame({'origin':new_origin, 'total_num':n_movies})
num_df.head()

,origin,total_num
0,Afghanistan,1
1,Argentina,3
2,Australia,162
3,Austria,12
4,Bahamas,5


In [536]:
major_origin = num_df[num_df.total_num >=100] 

In [537]:
major_origin

,origin,total_num
2,Australia,162
8,Canada,258
15,France,181
17,Germany,400
24,India,513
32,Japan,113
61,UK,492
62,USA,3192


In [538]:
major_origin['origin'].tolist()

['Australia', 'Canada', 'France', 'Germany', 'India', 'Japan', 'UK', 'USA']